In [19]:
import os
import matplotlib.pyplot as plt

from data_generator import DataGenerator
from dotenv import load_dotenv

# Load environment variables
load_dotenv();

In [20]:
from data_generator import DataGenerator

# Create train, validation, and test data generators
train_data_gen = DataGenerator(os.getenv('TRAIN_DATA_DIR'))
train_data_gen.summary()

val_data_gen = DataGenerator(os.getenv('VAL_DATA_DIR'))
val_data_gen.summary()

test_data_gen = DataGenerator(os.getenv('TEST_DATA_DIR'))
test_data_gen.summary()

data directory: ../data/raw/chest_xray/train/
class labels: ['PNEUMONIA', 'NORMAL']
number of data points: 5216
number of data points in class PNEUMONIA: 3875
number of data points in class NORMAL: 1341

data directory: ../data/raw/chest_xray/val/
class labels: ['PNEUMONIA', 'NORMAL']
number of data points: 16
number of data points in class PNEUMONIA: 8
number of data points in class NORMAL: 8

data directory: ../data/raw/chest_xray/test/
class labels: ['PNEUMONIA', 'NORMAL']
number of data points: 624
number of data points in class PNEUMONIA: 390
number of data points in class NORMAL: 234



In [21]:
import tensorflow as tf

# Load pretrained model
xception = tf.keras.applications.Xception(
    include_top=False
)
# Set weights in pretrained model to not trainable
xception.trainable = False

# Load pretrain model preprocessing function
xception_preproc_input = tf.keras.applications.xception.preprocess_input

# Set the preprocess functions in our data generators with xception_preprocess_input
train_data_gen.preproc_func = xception_preproc_input
val_data_gen.preproc_func = xception_preproc_input
test_data_gen.preproc_func = xception_preproc_input

In [22]:
from keras import Sequential, Input
from keras.layers import GlobalMaxPooling2D, Dense

# Store the number of classes
num_classes = len(train_data_gen.labels)

# Create model
model = Sequential(name='phneumonia_classifier')
model.add(Input((224,224,3)))
model.add(xception)
model.add(GlobalMaxPooling2D(name='global_pooling'))
model.add(Dense(num_classes, activation='softmax', name='output_layer'))
model.summary()

Model: "phneumonia_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_pooling                  │ (None, 2048)           │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,865,578 (79.60 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [23]:
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy

model.compile(
    loss=CategoricalCrossentropy(),
    optimizer=Adam(),
    metrics=[CategoricalAccuracy()]
)

In [ ]:
model.fit(
    x=train_data_gen,
    epochs=2,
    batch_size=train_data_gen.batch_size,
    validation_data=val_data_gen,
    validation_batch_size=val_data_gen.batch_size
)

In [18]:
model.evaluate(test_data_gen);

20/20 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - categorical_accuracy: 0.8237 - loss: 0.4102
